In [6]:
import pandas as pd

close_dm = pd.read_csv('card_data_(daesung,mega).csv', parse_dates=['transaction_date'])
close_dm = close_dm[['transaction_date', 'mandata_brand_name']]
m_df = close_dm[close_dm['mandata_brand_name'] == 'MEGASTUDYCO LTD_ALL']
d_df = close_dm[close_dm['mandata_brand_name'] == 'DIGITAL DAESUNG CO LTD_ALL']

주가 맞춤 저장

In [7]:
m_close = pd.read_csv('m_close.csv', parse_dates=['transaction_date'])
d_close = pd.read_csv('d_close.csv', parse_dates=['transaction_date'])

m_df = pd.merge(m_df, m_close[['transaction_date', 'Close']], on='transaction_date', how='left')
m_df = m_df.dropna()
d_df = pd.merge(d_df, d_close[['transaction_date', 'Close']], on='transaction_date', how='left')
d_df = d_df.dropna()

In [8]:
m_df = m_df.sort_values(by='transaction_date')
d_df = d_df.sort_values(by='transaction_date')

next_month_date = m_df['transaction_date'].iloc[-1] + pd.offsets.MonthBegin(1)

# 한 달치 데이터 생성
new_data = {
    'transaction_date': pd.date_range(start=next_month_date, periods=31),
    'mandata_brand_name': ['MEGASTUDYCO LTD_ALL'] * 31
}

# 한 달치 데이터 생성
new_data2 = {
    'transaction_date': pd.date_range(start=next_month_date, periods=31),
    'mandata_brand_name': ['DIGITAL DAESUNG CO LTD_ALL'] * 31
}

# 새로운 데이터 프레임 생성
new_rows_df = pd.DataFrame(new_data)
new_rows_df2 = pd.DataFrame(new_data2)

m_df = pd.concat([m_df, new_rows_df], ignore_index=True)
d_df = pd.concat([d_df, new_rows_df2], ignore_index=True)

,transaction_date,mandata_brand_name,Close
593,2023-06-27,DIGITAL DAESUNG CO LTD_ALL,6230.0
594,2023-06-28,DIGITAL DAESUNG CO LTD_ALL,6270.0
595,2023-06-29,DIGITAL DAESUNG CO LTD_ALL,6270.0
596,2023-06-30,DIGITAL DAESUNG CO LTD_ALL,6480.0
597,2023-07-01,DIGITAL DAESUNG CO LTD_ALL,NaN
598,2023-07-02,DIGITAL DAESUNG CO LTD_ALL,NaN
599,2023-07-03,DIGITAL DAESUNG CO LTD_ALL,NaN
600,2023-07-04,DIGITAL DAESUNG CO LTD_ALL,NaN
601,2023-07-05,DIGITAL DAESUNG CO LTD_ALL,NaN
602,2023-07-06,DIGITAL DAESUNG CO LTD_ALL,NaN


In [4]:
close_dm = pd.merge(d_df, m_df, on='transaction_date', how='outer', suffixes=('_d', '_m'))
close_dm.drop(['mandata_brand_name_d', 'mandata_brand_name_m'], axis=1, inplace=True)


close_dm.to_csv('stock_data_(daesung,mega).csv', index=False)

In [13]:
close_dm = pd.read_csv('stock_data_(daesung,mega).csv', index_col='transaction_date', parse_dates=True)

# 일별로 수익률을 계산
start_date = '2020-01-31'
end_date = '2020-11-30'

# 시작 날짜부터 끝 날짜까지의 데이터만 선택 sell_d 가 헷갈리지 않고 나을듯
sell_d = close_dm.loc[start_date:end_date]

# 각 기업의 일일수익률 계산
sell_d['close_d_return'] = sell_d['Close_d'].pct_change()
sell_d['close_m_return'] = sell_d['Close_m'].pct_change()

# 결측값을 이전 값으로 채우기
sell_d['close_d_return'] = sell_d['close_d_return'].ffill()
sell_d['close_m_return'] = sell_d['close_m_return'].ffill()

# 각 기업의 누적수익률 계산
sell_d['cumulative_d_return'] = (1 + sell_d['close_d_return']).cumprod() - 1
sell_d['cumulative_m_return'] = (1 + sell_d['close_m_return']).cumprod() - 1

# 수익률의 부호 변경
sell_d['d_return_sell'] = -sell_d['cumulative_d_return']

# close_h_return과 close_l_return을 더한 값을 return 열로 만들기
sell_d['return'] = sell_d['cumulative_m_return'] + sell_d['d_return_sell']

sell_d.to_csv('close_dm1.csv', index=True)

print("메가스터디 롱 누적수익률:", sell_d['cumulative_m_return'].iloc[-1]*100)
print("디지털대성 숏 누적수익률:", sell_d['d_return_sell'].iloc[-1]*100)
print("매매 누적수익률:", sell_d['return'].iloc[-1]*100)

return1 = sell_d['return'].iloc[-1]

In [17]:
# 일별로 수익률을 계산
start_date = '2020-11-30'
end_date ='2022-02-28'

# 시작 날짜부터 끝 날짜까지의 데이터만 선택
sell_m = close_dm.loc[start_date:end_date]

# 각 기업의 일일수익률 계산
sell_m['close_d_return'] = sell_m['Close_d'].pct_change()
sell_m['close_m_return'] = sell_m['Close_m'].pct_change()

# 결측값을 이전 값으로 채우기
sell_m['close_d_return'] = sell_m['close_d_return'].ffill()
sell_m['close_m_return'] = sell_m['close_m_return'].ffill()

# 각 기업의 누적수익률 계산
sell_m['cumulative_d_return'] = (1 + sell_m['close_d_return']).cumprod() - 1
sell_m['cumulative_m_return'] = (1 + sell_m['close_m_return']).cumprod() - 1

# 수익률의 부호 변경
sell_m['m_return_sell'] = -sell_m['cumulative_m_return']

# close_h_return과 close_l_return을 더한 값을 return 열로 만들기
sell_m['return'] = sell_m['cumulative_d_return'] + sell_m['m_return_sell']+return1

sell_m.to_csv('close_dm2.csv', index=True)

print("메가스터디 숏 누적수익률:", sell_m['m_return_sell'].iloc[-1]*100)
print("디지털대성 롱 누적수익률:", sell_m['cumulative_d_return'].iloc[-1]*100)
print("매매 누적수익률:", sell_m['return'].iloc[-1]*100)

return1 = sell_m['return'].iloc[-1]

주가저장코드

In [30]:
m_close = fdr.DataReader('072870', '2021-02-01', '2023-07-31') 
m_close = m_close[['Close']]
d_close = fdr.DataReader('068930', '2021-02-01', '2023-07-31') 
d_close = d_close[['Close']]
m_close.to_csv('m_close.csv', index=True)
d_close.to_csv('d_close.csv', index=True)

h_close = fdr.DataReader('069960', '2019-01-31', '2023-07-31') 
h_close = h_close[['Close']]
l_close = fdr.DataReader('023530', '2019-01-31', '2023-07-31') 
l_close = l_close[['Close']]
h_close.to_csv('h_close.csv', index=True)
l_close.to_csv('l_close.csv', index=True)

e_close = fdr.DataReader('139480', '2019-01-31', '2023-07-31') 
e_close = e_close[['Close']]
s_close = fdr.DataReader('004170', '2019-01-31', '2023-07-31') 
s_close = s_close[['Close']]
e_close.to_csv('e_close.csv', index=True)
s_close.to_csv('s_close.csv', index=True)